In [4]:
from openai import OpenAI
import os 

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [5]:
assistant_id = "asst_frbnNpiXlO88YArsHgiimoXI"

In [6]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from dateutil.parser import parse
from datetime import timedelta
from tzlocal import get_localzone
import tzlocal
import os

def add_event(start_time_str, summary, location=None, end_time_str=None, description=None, is_recurring=False, recurrence_rule=None):
    # Define the scope
    SCOPES = ['https://www.googleapis.com/auth/calendar']

    # Authenticate and create a service
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('calendar', 'v3', credentials=creds)

    # Parse the start time and handle the timezone
    start_time = parse(start_time_str)
    local_timezone = tzlocal.get_localzone_name()

    if not end_time_str:
        end_time = start_time + timedelta(minutes=15)
        end_time_str = end_time.isoformat()

    # Create the event
    event = {
        'summary': summary,
        'location': location,
        'description': description,
        'start': {
            'dateTime': start_time_str,
            'timeZone': local_timezone,
        },
        'end': {
            'dateTime': end_time_str,
            'timeZone': local_timezone,
        }
    }

    if is_recurring and recurrence_rule:
        event['recurrence'] = [recurrence_rule]

    # Add the event to the calendar
    created_event = service.events().insert(calendarId='primary', body=event).execute()
    print('Event created: %s' % (created_event.get('htmlLink')))

# Example usage of the function
add_event('2023-12-23T23:00:00', 'Meeting with Ahmed', 'Office')


Event created: https://www.google.com/calendar/event?eid=a2ZlM2pqNDVjc3JxOTg5dnJ2djhpb3E4a2MgdGFoYW0yMDIxNzE4QG0


In [7]:
assistant = client.beta.assistants.retrieve(
    assistant_id=assistant_id
)
assistant 

Assistant(id='asst_frbnNpiXlO88YArsHgiimoXI', created_at=1703344334, description=None, file_ids=[], instructions='You are a helpful assistant. If you are asked to add an event to the calendar, use the start time and the summary of the event with the provided add_events function to set an event in the google calender and let the user know that its done. ', metadata={}, model='gpt-3.5-turbo-1106', name='Calendar Assistant   ', object='assistant', tools=[ToolFunction(function=FunctionDefinition(name='add_event', description='Add event to Google calendar', parameters={'type': 'object', 'properties': {'start_time_str': {'type': 'string', 'description': 'The start time for the event in ISO 8601 format.'}, 'summary': {'type': 'string', 'description': 'A brief description or title of the event.'}, 'location': {'type': 'string', 'description': 'The location where the event will take place.'}, 'end_time_str': {'type': 'string', 'description': 'The end time for the event in ISO 8601 format.'}, 'd

In [8]:
thread = client.beta.threads.create()
thread

Thread(id='thread_iZ6pH7OZc6qAaDrgewAV5g5o', created_at=1703344533, metadata={}, object='thread')

In [10]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
    instructions="set an event for tomorrow at 3pm with Ahmed for coding practice, its going to be about 3 hours long and we will be using python"
)
run

Run(id='run_RsFH7WeIdLURji9aTutvpX6E', assistant_id='asst_frbnNpiXlO88YArsHgiimoXI', cancelled_at=None, completed_at=None, created_at=1703344656, expires_at=1703345256, failed_at=None, file_ids=[], instructions='set an event for tomorrow at 3pm with Ahmed for coding practice, its going to be about 3 hours long and we will be using python', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_iZ6pH7OZc6qAaDrgewAV5g5o', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='add_event', description='Add event to Google calendar', parameters={'type': 'object', 'properties': {'start_time_str': {'type': 'string', 'description': 'The start time for the event in ISO 8601 format.'}, 'summary': {'type': 'string', 'description': 'A brief description or title of the event.'}, 'location': {'type': 'string', 'description': 'The location where the event will take place.'}, 'end_time_str': 

In [11]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)


In [12]:
run

Run(id='run_RsFH7WeIdLURji9aTutvpX6E', assistant_id='asst_frbnNpiXlO88YArsHgiimoXI', cancelled_at=None, completed_at=None, created_at=1703344656, expires_at=1703345256, failed_at=None, file_ids=[], instructions='set an event for tomorrow at 3pm with Ahmed for coding practice, its going to be about 3 hours long and we will be using python', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_QtpjFqLHRaUx43rowHmQWlQP', function=Function(arguments='{"start_time_str": "2023-08-23T15:00:00", "summary": "Coding Practice with Ahmed", "location": "Virtual", "end_time_str": "2023-08-23T18:00:00", "description": "Python coding practice session with Ahmed.", "is_recurring": false}', name='add_event'), type='function'), RequiredActionFunctionToolCall(id='call_TRLvZNwOEaGa62eh3RwGFXyY', function=Function(arguments='{"start_time_str": "2023

In [13]:
import json 
# start_time_str, summary, location=None, end_time_str=None, description=None, is_recurring=False, recurrence_rule=None
def output(tool_call):
    start_time = json.loads(tool_call.function.arguments)["start_time_str"]
    summary = json.loads(tool_call.function.arguments)["summary"]
    add_event(start_time, summary)




tool_call = run.required_action.submit_tool_outputs.tool_calls
tool_outputs = map(output, tool_call)
tool_outputs = list(tool_outputs)
tool_outputs

Event created: https://www.google.com/calendar/event?eid=aDVscXBmNnRkMmIydjRtdm9rNXJpOHJrajggdGFoYW0yMDIxNzE4QG0
Event created: https://www.google.com/calendar/event?eid=N2ozdm45NzNpaGhvNjRjbzlxNXZxbGlwc2MgdGFoYW0yMDIxNzE4QG0


[None, None]

In [16]:
import json

def output(tool_call):
    # Assuming each tool_call item is a JSON string with the required arguments
    arguments = json.loads(tool_call.function.arguments)
    start_time = arguments["start_time_str"]
    summary = arguments["summary"]

    # Optional parameters are fetched with default values if they are not present
    location = arguments.get("location")
    end_time_str = arguments.get("end_time_str")
    description = arguments.get("description")
    is_recurring = arguments.get("is_recurring", False)
    recurrence_rule = arguments.get("recurrence_rule")

    add_event(start_time, summary, location, end_time_str, description, is_recurring, recurrence_rule)

# Assuming tool_call is a list of tool call objects
tool_call = run.required_action.submit_tool_outputs.tool_calls

# If you expect only one tool call and want to process only the first one
output(tool_call[0])

# If you need to handle multiple tool calls, use a for loop instead of map
# for call in tool_call:
#     output(call)

Event created: https://www.google.com/calendar/event?eid=dmtwZHBkcGpucWFob2gwYWRpNzJsN2dmYjggdGFoYW0yMDIxNzE4QG0
